In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

In [ ]:
from modules.retriever import ChromaRetriever
from modules.loaders import MyDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

COLLECTION_NAME = 'NDIS_ALL_PDFPLUMBER_TEXTS_1024_256'

loader = MyDirectoryLoader(dir_path = '../data')
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
search_kwargs = {
    "distance_metric": "cos",
    "fetch_k": 10, 
    "k": 20,
    "maximal_marginal_relevance": True
}
newCollection_kwargs = {
    "loader": loader,
    "splitter": splitter
}
retriever = ChromaRetriever(client_type='httpclient').build(COLLECTION_NAME, search_kwargs, newCollection_kwargs)

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    retriever, 
    "search_state_of_union",
    "Searches and returns documents regarding the state-of-the-union."
)
tools = [tool]

In [1]:
# https://python.langchain.com/docs/use_cases/question_answering/how_to/conversational_retrieval_agents

from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature = 0)

# This is needed for both the memory and the prompt
memory_key = "history"

memory = AgentTokenBufferMemory(memory_key=memory_key, llm=llm)



In [ ]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

system_message = SystemMessage(
        content=(
            "Do your best to answer the questions. "
            "Feel free to use any tools available to look up "
            "relevant information, only if neccessary"
        )
)

prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )

agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

from langchain.agents import AgentExecutor